**Analysis of Employee Travel Activities**


**Overview**

A company operating in the Northeastern United States has requested assistance in analyzing and visualizing some of their data related to the travel schedules and routes of their salespeople. The current state of the data requires that it be cleaned and filtered in order to be presented in ways which the company has requested. The following notebook demonstrates the steps taken to achieve this, while also allowing for modifications to be made to the filtering options to view different results.


The first step in this process involves importing all of the necessary packages that will be required to complete this analysis. The packages are listed below.

In [14]:
import os, re
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import ogr
from osgeo import osr
import zipfile
from datetime import datetime, timedelta

Next, the necessary variables are set up at the top of the notebook, allowing users to swap out different datasets and alter filtering options. 

***Input Variables:***

In [15]:
workspace = r"C:\Users\joefa\Documents\GEOG489\Lesson3\Lesson3Assignment\assignment3_data_March19"
employeeFile = "employees.csv"
citiesFile = "ne_cities.shp"
outputCSV = "locationsByEmployee.csv"
outputSHP = "travel_Line_jdf5716.shp"
employees = ['Jones, Frank', 'Brown, Justine', 'Samulson, Roger']
startDate = datetime(2016, 6, 26)    # Start and end dates are entered as datetime objects.
endDate = datetime(2017, 5, 11)      # To alter these objects, enter dates in the format (yyyy, mm, dd, (*optional hh, mm, ss))

First, the employee list is read into a Pandas DataFrame. This list features employee names, along with their work ID numbers.

In [16]:
employee_df = pd.read_csv(os.path.join(workspace, employeeFile), header = None, names = ['Name', 'ID'])
employee_df

,Name,ID
0,"Smith, Richard",1234421
1,"Moore, Lisa",1231233
2,"Jones, Frank",2132222
3,"Brown, Justine",2132225
4,"Samulson, Roger",3981232
5,"Madison, Margaret",1876541


We only want to pull data from relevant files. Since the workspace folder may contain other files with similar names, we set up a regular expression to filter through the folder and only create DataFrames from the correct files, which follow the format 'travel_####.csv'. These individual DataFrames are then concatenated into one single DataFrame. 

In [17]:
# This pattern will filter out any unwanted files (e.g., xyz_123.csv or travel_budgets.csv)
pattern = 'travel_\d\d\d\d\.csv$'

# The pattern is pre-compiled to save time and resources as it will be tested on multiple files
compiledRE = re.compile(pattern)

# CSV files that match the pattern are selected and loaded as DataFrames.
travelFiles = [pd.read_csv(os.path.join(workspace, file), sep = ',', header = None, names = ['ID', 'Start', 'End', 'Route'],
                           parse_dates = [1, 2], date_parser= lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 
               for file in os.listdir(workspace) if re.match(compiledRE, file)]

# The selected DataFrames are all added to one single DataFrame using the concat method.
travelFiles_df = pd.concat(travelFiles)
travelFiles_df.head()

C:\Users\joefa\anaconda3\envs\AC37\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  if __name__ == '__main__':


,ID,Start,End,Route
0,2132222,2016-01-07 16:00:00,2016-01-26 12:00:00,Cleveland;Bangor;Erie;Philadelphia;New York;Al...
0,1234421,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
0,2132225,2016-01-29 12:00:00,2016-02-03 15:00:00,Bangor;Cleveland;Augusta
0,2132225,2016-02-10 07:00:00,2016-02-22 22:00:00,Altoona;Augusta;Altoona;Bangor;Augusta;Columbus
0,2132222,2016-02-19 14:00:00,2016-02-25 13:00:00,Boston;Philadelphia


In order to match the travel routes to the employees, we join the two DataFrames on the employee IDs. This makes it possible to identify who was on which trip without having to cross reference the separate DataFrames.

In [18]:
# the merge method is used to join two DataFrames based on a field.
travelFiles_df_merged = employee_df.merge(travelFiles_df, left_on = 'ID', right_on = 'ID')
travelFiles_df_merged

,Name,ID,Start,End,Route
0,"Smith, Richard",1234421,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
1,"Smith, Richard",1234421,2016-04-09 06:00:00,2016-04-16 22:00:00,Augusta;New York;Columbus;Syracuse;Albany
2,"Smith, Richard",1234421,2016-04-22 15:00:00,2016-05-02 20:00:00,Columbus;Burlington;Augusta;Syracuse;Augusta;P...
3,"Smith, Richard",1234421,2016-05-09 06:00:00,2016-05-18 22:00:00,Boston;Albany;Harrisburg;Erie
4,"Smith, Richard",1234421,2016-05-19 08:00:00,2016-05-25 16:00:00,Portland;Washington
...,...,...,...,...,...
70,"Madison, Margaret",1876541,2017-03-05 11:00:00,2017-03-17 14:00:00,Altoona;Columbus;Philadelphia;Augusta;Bangor;B...
71,"Madison, Margaret",1876541,2017-04-11 15:00:00,2017-04-25 18:00:00,Portland;Burlington;Cleveland;Altoona;Scranton...
72,"Madison, Margaret",1876541,2017-05-20 15:00:00,2017-06-06 17:00:00,Albany;Boston;Pittsburgh;Cleveland;New York;Ph...
73,"Madison, Margaret",1876541,2018-01-05 10:00:00,2018-01-16 13:00:00,Philadelphia;Burlington;Washington;Erie


The next step is to create a copy of the DataFrame, perform our filtering operations on it (whose inputs were set up at the top of the workbook), orgranize the new DataFrame, and save it to a CSV file. 

In [19]:
# Only rows that meet the supplied filters will be added to the new DataFrame, which was copied from
# the original to avoid any disturbances to the file. 
filteredTravelFiles_df = travelFiles_df_merged[(travelFiles_df_merged.Start >= startDate) & 
                                              (travelFiles_df_merged.End <= endDate) & 
                                              (travelFiles_df_merged.Name.isin(employees))].copy()


# A new field, 'Duration', is added to the new DataFrame. This field is calculated by subtracting
# the end date from the start date, using the datetime module. The columns are then reorganized
# and finally the DataFrame is exported to a CSV file, using the to_csv() function. 
filteredTravelFiles_df['Duration'] = ((filteredTravelFiles_df.End - filteredTravelFiles_df.Start).dt.days)
filteredTravelFiles_df = filteredTravelFiles_df[['Name', 'ID', 'Duration', 'Start', 'End', 'Route']].sort_values(['Name', 'Duration', 'Start', 'End']).reset_index(drop = True)
filteredTravelFiles_df.to_csv(os.path.join(workspace, outputCSV))
filteredTravelFiles_df

,Name,ID,Duration,Start,End,Route
0,"Brown, Justine",2132225,5,2016-11-30 16:00:00,2016-12-06 14:00:00,Pittsburgh;Syracuse
1,"Brown, Justine",2132225,9,2016-12-12 12:00:00,2016-12-21 19:00:00,Columbus;Boston;Pittsburgh
2,"Brown, Justine",2132225,12,2016-08-02 13:00:00,2016-08-14 16:00:00,Albany;Scranton;Philadelphia;Scranton;Augusta
3,"Brown, Justine",2132225,12,2017-01-07 11:00:00,2017-01-19 17:00:00,Harrisburg;Portland;Boston;Syracuse;Albany
4,"Brown, Justine",2132225,17,2017-03-27 06:00:00,2017-04-13 14:00:00,Portland;Boston;Bangor;Washington;Harrisburg;P...
5,"Brown, Justine",2132225,19,2016-11-04 17:00:00,2016-11-23 20:00:00,New York;Portland;Boston;Portland;Washington;P...
6,"Jones, Frank",2132222,5,2016-07-10 18:00:00,2016-07-15 18:00:00,Harrisburg;Augusta
7,"Jones, Frank",2132222,10,2017-01-18 12:00:00,2017-01-28 19:00:00,Philadelphia;Boston;Altoona;Harrisburg;Scranton
8,"Jones, Frank",2132222,17,2017-03-16 17:00:00,2017-04-02 18:00:00,Augusta;New York;Scranton;Columbus;Washington;...
9,"Samulson, Roger",3981232,7,2016-10-15 07:00:00,2016-10-22 17:00:00,Boston;Syracuse;Portland;Altoona


### **Creating a shapefile**

While the company wanted to have their data organized in tabular form, they also requested to be able visualize these trips on a map. For this step, we upload a city point shapefile using geopandas to match the cities' coordinates with the trip DataFrame. 

In [20]:
# The geopandas function read_file() is used to read the city point shapefile to a DataFrame.
cityData = gpd.read_file(os.path.join(workspace, citiesFile))
cityData

,OBJECTID,UIDENT,POPCLASS,NAME,CAPITAL,STATEABB,COUNTRY,geometry
0,15,62507.0,2.0,Bangor,-1.0,US-ME,USA,POINT (-68.77619 44.81189)
1,20,64707.0,2.0,Waterville,-1.0,US-ME,USA,POINT (-69.62073 44.53724)
2,23,65607.0,2.0,Augusta,1.0,US-ME,USA,POINT (-69.78141 44.32166)
3,26,66707.0,2.0,Berlin,-1.0,US-NH,USA,POINT (-71.19324 44.46470)
4,29,67407.0,2.0,Lewiston,-1.0,US-ME,USA,POINT (-70.19406 44.10972)
...,...,...,...,...,...,...,...,...
132,406,121107.0,2.0,Hickory,-1.0,US-NC,USA,POINT (-81.36642 35.72144)
133,413,121907.0,3.0,Knoxville,-1.0,US-TN,USA,POINT (-83.93407 35.98019)
134,414,122007.0,2.0,Oak Ridge,-1.0,US-TN,USA,POINT (-84.25911 36.00918)
135,421,122707.0,2.0,Asheville,-1.0,US-NC,USA,POINT (-82.54221 35.60217)


Coordinates are extracted from the city DataFrame and added to tuples which align with the various routes. These tuples are joined to the main DataFrame to allow a line shapefile to be created. 

In [21]:
# A list comprehension is used to parse through the route strings and search for each city's accompanying 
# coordinates. This is then converted to a DataFrame and joined to the main DataFrame. 
wkt = [ 'LineString (' + ','.join([ '{0} {1}'.\
    format(cityData[cityData.NAME == city].geometry.x.iloc[0],\
           cityData[cityData.NAME == city].geometry.y.iloc[0]) \
           for city in r.split(';') ])+')' for r in filteredTravelFiles_df.Route]

wkt_df = pd.DataFrame(wkt, columns = ['linestring'])

travelFilesLinestring_df = filteredTravelFiles_df.join(wkt_df)
travelFilesLinestring_df



,Name,ID,Duration,Start,End,Route,linestring
0,"Brown, Justine",2132225,5,2016-11-30 16:00:00,2016-12-06 14:00:00,Pittsburgh;Syracuse,LineString (-79.91439598338775 40.447399277754...
1,"Brown, Justine",2132225,9,2016-12-12 12:00:00,2016-12-21 19:00:00,Columbus;Boston;Pittsburgh,LineString (-82.98182532058195 39.972384950705...
2,"Brown, Justine",2132225,12,2016-08-02 13:00:00,2016-08-14 16:00:00,Albany;Scranton;Philadelphia;Scranton;Augusta,LineString (-73.78191599152962 42.674015577157...
3,"Brown, Justine",2132225,12,2017-01-07 11:00:00,2017-01-19 17:00:00,Harrisburg;Portland;Boston;Syracuse;Albany,LineString (-76.82629734234831 40.266819768117...
4,"Brown, Justine",2132225,17,2017-03-27 06:00:00,2017-04-13 14:00:00,Portland;Boston;Bangor;Washington;Harrisburg;P...,LineString (-70.27758484016806 43.698554552950...
5,"Brown, Justine",2132225,19,2016-11-04 17:00:00,2016-11-23 20:00:00,New York;Portland;Boston;Portland;Washington;P...,LineString (-73.95605368612854 40.648657910034...
6,"Jones, Frank",2132222,5,2016-07-10 18:00:00,2016-07-15 18:00:00,Harrisburg;Augusta,LineString (-76.82629734234831 40.266819768117...
7,"Jones, Frank",2132222,10,2017-01-18 12:00:00,2017-01-28 19:00:00,Philadelphia;Boston;Altoona;Harrisburg;Scranton,LineString (-75.16727846213115 39.941228213611...
8,"Jones, Frank",2132222,17,2017-03-16 17:00:00,2017-04-02 18:00:00,Augusta;New York;Scranton;Columbus;Washington;...,LineString (-69.78141264717578 44.321655641294...
9,"Samulson, Roger",3981232,7,2016-10-15 07:00:00,2016-10-22 17:00:00,Boston;Syracuse;Portland;Altoona,LineString (-71.08008355451227 42.362833160122...


Next, a new shapefile is created and saved under a variable name set at the top of the notebook. Attributes of the shapefile include the employee's name, the route, and the duration of the trip. This information is matched with a line that traces the trip through its various stops. 

In [22]:
sr = osr.SpatialReference()   # create spatial reference object
sr.ImportFromEPSG(4326)       # set it to EPSG:4326
drv = ogr.GetDriverByName('ESRI Shapefile')
outfile = drv.CreateDataSource(os.path.join(workspace, outputSHP)) 
outlayer = outfile.CreateLayer(outputSHP.replace('.shp', ''), geom_type=ogr.wkbLineString, srs = sr)  # create new layer in the shapefile 
 
nameField = ogr.FieldDefn('Name', ogr.OFTString)        # create new field of type string called Name to store the employee names
outlayer.CreateField(nameField)                         # add this new field to the output layer
nameField = ogr.FieldDefn('Route', ogr.OFTString)       # create new field of type string called Route to store the route strings
outlayer.CreateField(nameField)                         # add this new field to the output layer
nameField = ogr.FieldDefn('Duration', ogr.OFTInteger)   # create new field of type integer called Duration to store the duration numbers
outlayer.CreateField(nameField)                         # add this new field to the output layer

# add this new field to the output layer
 
featureDefn = outlayer.GetLayerDefn()


for row in travelFilesLinestring_df.itertuples():        # loop through DataFrame with the itertuples method
    ingeom = row.linestring                              # get geometry of feature from the linstring tuple
    outgeom = ogr.CreateGeometryFromWkt(ingeom)          # Create geometry from wkt using the linstring tuples
 
    outFeature = ogr.Feature(featureDefn)                # create a new feature
    outFeature.SetGeometry(outgeom)                      # set its geometry to outgeom
    outFeature.SetField('Name', row.Name)                # set the feature's Name field to the Name value of the DataFrame
    outFeature.SetField('Route', row.Route)              # set the feature's Route field to the Route value of the DataFrame
    outFeature.SetField('Duration', row.Duration)        # set the feature's Duration field to the Duration value of the DataFrame 
    outlayer.CreateFeature(outFeature)                   # finally add the new output feature to outlayer
    outFeature = None
 
outfile = None         # close output file

In order to visualize these travel routes, we will zip the shapefile and upload it to ArcGIS Online using the Esri API. This will be viewable through the map widget blow.

In [23]:
# function to create zipped shapefile for a given filename without extension
def zipShapefile(name):
    compiledRE = re.compile(name+'(?!.zip)\....$')
    with zipfile.ZipFile( os.path.join(workspace, name + '.zip'), 'w', zipfile.ZIP_DEFLATED) as zf:  # create zipfile
        for file in os.listdir(workspace):                                                           # go through files in workspace
            if compiledRE.match(file):                                                               # test whether file is part of the shapefile to be zipped
                zf.write(os.path.join(workspace,file),file,zipfile.ZIP_DEFLATED)                     # add file to zipfile
                
# create zipped version of observation shapefile
zipShapefile( os.path.splitext(outputSHP)[0] )

The following code connects this notebook to ArcGIS. Adjustments may need to be made to the filepaths, depending on where ArcGIS is stored on your local machine. 

In [24]:
import os,sys
os.environ["PATH"] = r"{};{}".format(os.environ["PATH"], r"C:\Program Files\ArcGIS\Pro\bin")
sys.path.append(r"C:\Program Files\ArcGIS\Pro\Resources\ArcPy")
import arcgis
from arcgis.gis import GIS
gis = GIS('pro')

The shapefile is published on ArcGIS Online and ready for viewing. 

In [25]:
# Add the shapefile to ArcGIS Online
travelRoutes = gis.content.add({'type': 'Shapefile'}, os.path.join(workspace, outputSHP.replace('.shp','.zip')))

# Publish the layer in ArcGIS Online 
travelRoutesFS = travelRoutes.publish()

In [26]:
# Set up the map
travelMap = gis.map()

# Add the published layer to the map
travelMap.add_layer(travelRoutesFS, {})

# View the map with the layer
travelMap

MapView(layout=Layout(height='400px', width='100%'))

# Conclusion

This notebook takes travel data from a company operating in the Northeastern United States. Their original datasets were scattered throughout their organization and did not contain all of the necessary information to perform efficient analysis and make important business decisions. This notebook transformed the data to make it more accessible, filterable, and presentable. Users can adjust data sources and filtering option at the top of the workbook (in the *input variables* section) to perform similar tasks and answer any further questions they may have about the data and the travelling activity at their company. 